In [ ]:
from collections import defaultdict

import numpy as np
import pandas as pd
import torch
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from transformers import AutoTokenizer
from transformers import logging

import src
from src.bert import module
from src.bert import training
from src.bert.dataset import PBertDataset
from src.bert.dataset import strategies

Best Youden thresh: {0: 0.5013, 1: 0.0728, 2: 0.0852, 3: 0.0292}
Best BF thresh {0: 0.5, 1: 0.5, 2: 0.45, 3: 0.23}
epoch=13 train_loss=0.0351 eval_loss=0.1581 score=0.7092
{0: 0.5013018, 1: 0.5017193, 2: 0.42243505, 3: 0.38281676}

In [ ]:
logging.set_verbosity_error()

# model hyper-parameters
LR = 4e-6
N_EPOCHS = 13
BATCH_SIZE = 16

TOKENIZER = "deepset/gbert-large"
BASE_MODEL = "deepset/gbert-large"

STRATEGY = strategies.MLMin1PopIdeol(output_fmt="single_task")

DEVICE = "cuda"

TRESHOLDS = {0: 0.5013018, 1: 0.5017193, 2: 0.42243505, 3: 0.38281676}

In [ ]:
dataset = PBertDataset.from_disk(src.PATH / "data/bert/dataset.csv.zip", label_strategy=STRATEGY)

In [ ]:
len(dataset)

8795

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)
collate_fn = dataset.create_collate_fn(tokenizer)

In [ ]:
splits = KFold(n_splits=5, shuffle=True, random_state=42)

results = []
for fold, (train_idx, test_idx) in enumerate(splits.split(np.arange(len(dataset))), 1):
    print("#" * 50)
    print(f"{fold=}")

    # create data
    train_loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        collate_fn=collate_fn,
        sampler=SubsetRandomSampler(train_idx),
    )
    test_loader = DataLoader(
        dataset,
        batch_size=32,
        collate_fn=collate_fn,
        sampler=SubsetRandomSampler(test_idx),
    )

    model = module.BertSingleTaskMultiLabel(num_labels=dataset.num_labels, name=BASE_MODEL)
    model.train()
    model = model.to(DEVICE)
    model.set_seed(seed=10)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=LR,
        amsgrad=False,
        weight_decay=1e-2,
    )

    lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=N_EPOCHS,
        eta_min=1e-9,
    )

    for epoch in range(1, N_EPOCHS + 1):
        train_loss = training.train_epoch(model, train_loader, optimizer, lr_scheduler)
        eval_loss, score, _ = training.eval_epoch(model, test_loader)
        print(f"{epoch=} {train_loss=:.4f} {eval_loss=:.4f} {score=:.4f}")

    with torch.inference_mode():
        y_true = []
        y_pred = []
        for batch in test_loader:
            encodings = batch["encodings"]
            encodings = encodings.to(DEVICE)
            labels = batch["labels"].to(DEVICE)
            preds = model.predict_proba(encodings)
            y_true.extend(batch["labels"].numpy())
            y_pred.extend(preds)
        y_pred_05 = np.where(np.array(y_pred) > 0.5, 1, 0)
        y_pred_thresh = model.apply_thresh(np.array(y_pred), TRESHOLDS)
        y_true = np.array(y_true)

    print("THRESH .5")
    print(classification_report(y_true, y_pred_05, target_names=dataset.labels, zero_division=0))
    print()
    print("THRESH DICT")
    print(
        classification_report(y_true, y_pred_thresh, target_names=dataset.labels, zero_division=0)
    )
    classification_dict = classification_report(
        y_true, y_pred_thresh, target_names=dataset.labels, zero_division=0, output_dict=True
    )
    results.append(classification_dict)
    out = pd.DataFrame(classification_dict)
    out.to_csv(src.PATH / f"results/kfold/fold_{fold}.csv")

##################################################
fold=1


Best BF thresh {0: 0.65, 1: 0.25, 2: 0.32, 3: 0.23}
epoch=1 train_loss=0.4465 eval_loss=0.3465 score=0.3847


Best BF thresh {0: 0.45, 1: 0.4, 2: 0.34, 3: 0.18}
epoch=2 train_loss=0.2749 eval_loss=0.2499 score=0.5457


Best BF thresh {0: 0.44, 1: 0.44, 2: 0.44, 3: 0.17}
epoch=3 train_loss=0.2413 eval_loss=0.1475 score=0.6354


Best BF thresh {0: 0.52, 1: 0.37, 2: 0.24, 3: 0.24}
epoch=4 train_loss=0.1540 eval_loss=0.2180 score=0.6863


...

Best BF thresh {0: 0.14, 1: 0.25, 2: 0.4, 3: 0.49}
epoch=10 train_loss=0.1307 eval_loss=0.1921 score=0.7153


Best BF thresh {0: 0.08, 1: 0.27, 2: 0.49, 3: 0.11}
epoch=11 train_loss=0.0519 eval_loss=0.2505 score=0.6882


Best BF thresh {0: 0.25, 1: 0.47000000000000003, 2: 0.67, 3: 0.18}
epoch=12 train_loss=0.1028 eval_loss=0.3465 score=0.7080


Best BF thresh {0: 0.19, 1: 0.28, 2: 0.45, 3: 0.5700000000000001}
epoch=13 train_loss=0.0694 eval_loss=0.3839 score=0.7167


THRESH .5
              precision    recall  f1-score   support

       elite       0.80      0.88      0.84       640
       centr       0.66      0.66      0.66       316
        left       0.65      0.79      0.71       261
       right       0.68      0.63      0.65       140

   micro avg       0.73      0.79      0.76      1357
   macro avg       0.70      0.74      0.72      1357
weighted avg       0.73      0.79      0.75      1357
 samples avg       0.38      0.39      0.38      1357


THRESH DICT
              precision    recall  f1-score   support

       elite       0.80      0.88      0.84       640
       centr       0.66      0.66      0.66       316
        left       0.64      0.81      0.71       261
       right       0.64      0.66      0.65       140

   micro avg       0.72      0.79      0.76      1357
   macro avg       0.69      0.75      0.72      1357
weighted avg       0.72      0.79      0.75      1357
 samples avg       0.38      0.39      0.38      1357


In [ ]:
score_dict = defaultdict(list)
for result in results:
    for dim, scores in result.items():
        score_dict[dim].append(scores["f1-score"])

out_data = []
for dim, scores in score_dict.items():
    out = {"dim": dim, "mean": np.mean(scores), "std": np.std(scores)}
    out_data.append(out)

In [ ]:
pd.DataFrame(out_data)

,dim,mean,std
0,elite,0.846780,0.007444
1,centr,0.695982,0.018900
2,left,0.713261,0.009890
3,right,0.674466,0.030877
4,micro avg,0.766746,0.006838
5,macro avg,0.732623,0.010373
6,weighted avg,0.766799,0.007123
7,samples avg,0.392949,0.011762
